# Retargeting Pascal0 for Intel 64-bit Architecture
Final project for COMPSCI 4TB3: Syntax-Based Tools and Compilers

By James Priebe, McMaster University

Email: priebejp@mcmaster.ca

Student number: 1135001


## Introduction
The topic of this report is a new compiler backend which generates NASM-syntax 64 bit assembly code. Included are a summary of the changes made to account for differences from the original MIPS architecture as well as the full code (CGx86.py and P0.py; scanner and symbol table unchanged from original compiler) and test suite used.

### Changes from Proposal
This report was created using Jupyter, a live code documentation tool, rather than the literate programming tool noweb as originally planned.

## Calling Convention
The generated assembly code handles procedure calls similarly to MIPS. It adheres to the C 64-bit convention only for standard procedure calls printf and scanf (see section: Input and Output).

### Caller
The caller pushes arguments to the stack, and is responsible for cleaning the stack upon return. Every time an actual parameter is pushed, we increment the global variable stacksize, which is set back to zero after the call. Parameters are generated by the following function:

In [ ]:
def genActualPara(ap, fp, n):
    """Pass parameter, ap is actual parameter, fp is the formal parameter,
    either Ref or Var, n is the parameter number"""
    global stacksize

    if type(fp) == Ref:  #  reference parameter, assume p is Var

        if ap.adr != 0:  #  load address in register
            r = obtainReg()
            loadAddress(r, ap.reg, ap.adr)
        else: 
            r = ap.reg  #  address already in register
        
        putInstr('push ' + r)
        stacksize += 8 # we restore stack after return
        releaseReg(r)

    else:  #  value parameter
        if type(ap) != Cond:
            if type(ap) != Reg: 
                ap = loadItem(ap)
            putInstr('push ' + ap.reg)
            stacksize += 8
            releaseReg(ap.reg)
        else: mark('unsupported parameter type')

And the procedure call:

In [ ]:
def genCall(pr):
    """Assume pr is Proc"""
    global stacksize

    putInstr('call ' + pr.name)
    putInstr('add rsp, ' + str(stacksize)) #clean the stack
    stacksize = 0

Here is an example of generated code:

In [ ]:
main:

    ...

    mov r12, 7
    push r12
    lea r15, [x_]
    push r15
    call q
    add rsp, 16
    
    ...

### Callee
The callee is then responsible for creating a stack frame and making space for local variables. The stack has the following layout (RBP is the frame pointer register):

Arguments are stored in **reverse order** at RBP + 16, RBP + 24, ...
Local variables are store in **regular order** at RBP - 8, RBP - 16, ...

Here is an example demonstrating this:

In [ ]:
q:
    push rbp              
    mov rbp, rsp         #set up stack frame

    sub rsp, 32          #allocate space for 4 local variables
    mov r13, [40 + rbp]  #load the first argument (second, third, fourth at RBP + 32, RBP + 24, RBP +16)
    mov r9, [0 + r13]
    mov [-8 + rbp], r9   #store argument 1 in local variable 1

## Input and Output
x86 does not support printing integers directly using a system call. 
The compiler generates calls to the external c functions *printf* and *scanf*. We must adhere to the C 64 bit calling convention, which is different from 32 bit. The first 6 arguments must be stored in registers
RDI, RSI, RDX, RCX, R8, and R9. Additionally, printf and scanf require the number of floating-point arguments to be stored in RAX, which in our case is always zero.

Upon program entry the compiler generates the following boilerplate for reading integer and newlines and writing integers:

In [ ]:
    extern printf
    extern scanf
    global main
 
    section .data

newline:    db "", 10, 0
write_msg:  db "%d", 10, 0
read_msg:   db "Enter an integer: ", 0
read_format:    db "%d", 0


And the generator functions:

In [ ]:
def genWrite(x):

    putInstr('')
    putInstr('mov rdi, write_msg')
    loadItemReg(x, 'rsi')
    putInstr('mov rax, 0')
    putInstr('call printf')
    putInstr('')


def genWriteln():

    putInstr('')
    putInstr('mov rdi, newline')
    putInstr('mov rax, 0')
    putInstr('call printf')
    putInstr('')

def genRead(x):
    """Assume x is Var"""

    putInstr('')
    putInstr('mov rdi, read_msg')
    putInstr('mov rax, 0')
    putInstr('call printf')
    putInstr('mov rdi, read_format')
    putInstr('mov rsi, number')
    putInstr('mov rax, 0')
    putInstr('call scanf')
    putInstr('mov rsi, [number]')
    putInstr('mov [' + str(x.adr) + '], rsi')
    putInstr('')

## Register Usage
x_86 64 introduces eight new general-purpose registers, r8 - r15.
This greatly simplifies translation from MIPS, because the original "general purpose" registers, RDX, RSI, etc are not truly general purpose; for example they are used to store the results of division and comparison operations. Thus we use RCX and R8 - R15 as direct analogs of MIPS $t0 - $t8. Our compiler uses the other registers for the following special purposes:

**RBX**
Zero register. RBX is not nonvolatile. It can be set to zero at the beginning of main and will not be destroyed by procedure calls.

**RAX**
Number of floating point arguments for printf and scanf calls.
Also holds the quotient of div operation.

**RDX**
Holds remainder of div instruction. Used for modulo
Contains the format string for scanf and printf

**RSI**
Read/Write argument for printf/scanf

**RDI**
Format string for printf/scanf

## Addressing Mode
The compiler uses the following addressing modes, with example instructions:

**Register**
mov r8, r10
mul r8, r10

**Immediate**
mov rax, 0
add rsi, 8

**Direct/offset addressing**
lea r9, [x_ + 32]
mov [x_], r10 

For direct addressing the range is extended to -2^40 to 2^40 -1; the theoretical range of -2^64 to 2^64 -1 is not supported by current CPUs.

## Conditional Branching
Branching requires an additional operation from MIPS. The branch only takes one operation, the target address/label. First a *cmp* instruction must be generated, which sets the EFLAGS register that is used by the branch instruction. Below are the functions for generating conditional branches:

In [ ]:
def condOp(cd):
    """Assumes op in {EQ, NE, LT, LE, GT, GE}, return instruction mnemonic"""
    return 'je' if cd == EQ else \
           'jne' if cd == NE else \
           'jl' if cd == LT else \
           'jle' if cd == LE else \
           'jg' if cd == GT else \
           'jge'

def genCond(x):
    """Assume x is Bool, generate code for branching on x"""
    if type(x) != Cond: x = loadBool(x)
    neg = condOp(negate(x.cond))
    putInstr('cmp ' + x.left + ', ' + x.right)
    putInstr(neg + ' ' + x.labA[0])
    releaseReg(x.left); releaseReg(x.right); putLab(x.labB)
    return x

## Other Changes
#### Two-Operand Instructions
All instructions used by the compiler have at most two operands. Most arithmetic operations must be done in two steps, i.e, the x86 equivalent for the MIPS instruction

sub \$fp, \$sp, 4

is

mov rbp, rsp
sub rbp, 4

However many operations are of the form *x = x + y*, which can be done in a single instruction.
#### Division and modulus
Division and modulus are single-operand, register-only instructions in x86. The *idiv* instruction takes the divisor operand and stores the quotient in RAX and RDX, respectively. Below is the code generator for a modulus operation:

In [ ]:
def putModulo(x, y):
    if type(x) != Reg: x = loadItem(x)
    if x.reg == R0: x.reg, r = obtainReg(), R0
    else: r = x.reg # r is source, x.reg is destination
    if type(y) == Const:
        testRange(y) 
        putInstr('mov rax, ' + r)
        yc = obtainReg()
        putInstr('mov ' + yc + ', ' + str(y.val))
        putInstr('xor rdx, rdx')
        putInstr('idiv ' + yc)
        releaseReg(yc)
        # remainder is stored in rdx
        putInstr('mov ' + r + ', rdx')

    else:
        if type(y) != Reg: y = loadItem(y)
        putInstr('mov ' + x.reg + ', ' + r)
        putInstr('mov rax, ' + x.reg)
        putInstr('xor rdx, rdx')
        putInstr('idiv ' + y.reg)
        putInstr('mov ' + x.reg + ', rdx')
        releaseReg(y.reg)
    return x

#### Other files
P0.py has been modified to set the size of types Array and Boolean to 8 instead of 4.

## Testing
Comprehensively testing a compiler for correctness is large and difficult undertaking. Compilers of well-known languages can be tested by compiling a suitable large open-source project and running its own test suite. The Pascal0 compiler was tested for completeness with several programs designed to test specific attributes of the language, as well as with a factorial program for more robust coverage. Actual output matches expected output for all test cases. See Appendix C for the full list of test programs, annotated with their expected output. 

## Running the Compiler
The NASM compiler is required to compile and link the generated assembly code into an executable binary. A script, compile.py, has been included which will generate the assembler code and compile a 64-bit binary using NASM. Execute the script from the same directory as the compiler files as follows:

In [ ]:
python3 compile.py path/to/myfile.p

# Produces
path/to/myfile.s  #generated code
path/to/myfile    #executable

## Appendix A: Code Generator and P0.py

In [20]:
# %load CGx86.py
"""
Pascal0 Code Generator for x86_64. 
A retargetting of the MIPS Code Generator by Emil Sekerinski, March 2016.
Using delayed code generation for a one-pass compiler. The types of symbol
table entries for expressions, Var, Ref, Const, are extended by two more
types Reg for expression results in a register, and Cond, for short-circuited
Boolean expressions with two branch targets.

"""

import SC  #  used for SC.error
from SC import TIMES, DIV, MOD, AND, PLUS, MINUS, OR, EQ, NE, LT, GT, LE, \
     GE, NOT, mark
from ST import Var, Ref, Const, Type, Proc, StdProc, Int, Bool, Array, Record


# using rbx as zero register
R0 = 'rbx'; FP = 'rbp'; SP = 'rsp'  # reserved registers

# track size of parameters pushed to stack 
# so it can be restored after procedure return
global stacksize
stacksize = 0

class Reg:
    """
    For integers or booleans stored in a register;
    register can be $0 for constants '0' and 'false'
    """
    def __init__(self, tp, reg):
        self.tp, self.reg = tp, reg

class Cond:
    """
    For a boolean resulting from comparing left and right by cond:
    left, right are either registers or constants, but one has to be a register;
    cond is one of 'EQ', 'NE', 'LT', 'GT', 'LE', 'GE';
    labA, labB are lists of branch targets for when the result is true or false
    if right is $0, then cond 'EQ' and 'NE' can be used for branching depending
    on register left.
    """
    count = 0
    def __init__(self, cond, left, right):
        self.tp, self.cond, self.left, self.right = Bool, cond, left, right
        self.labA = ['C' + str(Cond.count)]; Cond.count += 1
        self.labB = ['C' + str(Cond.count)]; Cond.count += 1

# asm is the string with the generated assembly code
# curlev is the current level of nesting of procedures
# regs is the set of available registers for expression evaluation

def init():
    """initializes the code generator"""
    global asm, curlev, regs
    asm, curlev = '', 0
    # x86_64 registers, equivalent to MIPs t1..t8
    regs = {'rcx', 'r8', 'r9', 'r10', 'r11', 'r12', 'r13', 'r14', 'r15'} 
                                

def obtainReg():
    if len(regs) == 0: mark('out of registers'); return R0
    else: return regs.pop()

def releaseReg(r):
    if r not in (R0, SP, FP): regs.add(r)

def putLab(lab, instr = ''):
    """Emit label lab with optional instruction; lab may be a single
    label or a list of labels"""
    global asm
    if type(lab) == list:
        for l in lab[:-1]: asm += l + ':\n'
        asm += lab[-1] + ':\t' + instr + '\n'
    else: asm += lab + ':\t' + instr + '\n'

def putInstr(instr):
    """Emit an instruction"""
    global asm
    asm += ('\t' + instr + '\n')


# emit two op
def put(op, a, b):
    putInstr(op + ' ' + a + ', ' + str(b))


def loadAddress(reg, offset, address):
    if offset == R0: 
        putInstr('lea ' + reg + ', [' + str(address) + ']')
    else: 
        putInstr('lea ' + reg + ', [' + str(address) + ' + ' + str(offset) + ']')

def store(reg, offset, address):
    if offset == R0: 
        putInstr('mov [' + str(address) + '], ' + reg)
    else:
        putInstr('mov [' + str(address) + ' + ' + str(offset) + '], ' + reg)

def load(reg, offset, address):
    if offset == R0: 
        putInstr('mov ' + reg + ', [' + str(address) + ']')
    else: 
        putInstr('mov ' + reg + ', [' + str(address) + ' + ' + str(offset) + ']')

#put constant in register
def moveConst(r, val):
    putInstr('mov ' + r + ', '+ str(val))


def testRange(x):
    """Check if x is suitable for immediate addressing"""
    if x.val >= 0x10000000000 or x.val < -0x10000000000: mark('value too large')
    
def loadItemReg(x, r):
    """Assuming item x is Var, Const, or Reg, loads x into register r"""
    if type(x) == Var:
        load(r, x.reg, x.adr); releaseReg(x.reg)         
    elif type(x) == Const:
        testRange(x); moveConst(r, x.val)
    elif type(x) == Reg: # move to register r
        putInstr('mov ' + r + ', ' + x.reg)
    else: assert False

def loadItem(x):
    """Assuming item x is Var or Const, loads x into a new register and
    returns a new Reg item"""
    if type(x) == Const and x.val == 0: r = R0 # use R0 for "0"
    else: r = obtainReg(); loadItemReg(x, r)
    return Reg(x.tp, r)

def loadBool(x):
    """Assuming x is Var or Const and x has type Bool, loads x into a
    new register and returns a new Cond item"""
    # improve by allowing c.left to be a constant
    if type(x) == Const and x.val == 0: r = R0 # use R0 for "false"
    else: r = obtainReg(); loadItemReg(x, r)
    c = Cond(NE, r, R0)
    return c

def putOp (cd, x, y):
    """For operation op with mnemonic cd, emit code for x op y, assuming
    x, y are Var, Const, Reg"""
    if type(x) != Reg: x = loadItem(x)
    if x.reg == R0: x.reg, r = obtainReg(), R0
    else: r = x.reg # r is source, x.reg is destination
    if type(y) == Const:
        testRange(y) 
        putInstr('mov ' + r + ', ' + x.reg)
        putInstr(cd + ' ' + r + ', ' + str(y.val))
    else:
        if type(y) != Reg: y = loadItem(y)
        putInstr('mov ' + x.reg + ', ' + r)
        putInstr(cd + ' ' + x.reg + ', ' + y.reg)
        releaseReg(y.reg)
    return x


def putDivide(x, y):
    if type(x) != Reg: x = loadItem(x)
    if x.reg == R0: x.reg, r = obtainReg(), R0
    else: r = x.reg # r is source, x.reg is destination
    if type(y) == Const:
        testRange(y) 
        putInstr('mov rax, ' + r)
        yc = obtainReg()
        putInstr('mov ' + yc + ', ' + str(y.val))
        putInstr('xor rdx, rdx')
        putInstr('idiv ' + yc)
        releaseReg(yc)
        # retrieve result from rax
        putInstr('mov ' + r + ', rax')

    else:
        if type(y) != Reg: y = loadItem(y)
        putInstr('mov ' + x.reg + ', ' + r)
        putInstr('mov rax, ' + x.reg)
        putInstr('xor rdx, rdx')
        putInstr('idiv ' + y.reg)
        putInstr('mov ' + x.reg + ', rax')
        releaseReg(y.reg)
    return x


def putModulo(x, y):
    if type(x) != Reg: x = loadItem(x)
    if x.reg == R0: x.reg, r = obtainReg(), R0
    else: r = x.reg # r is source, x.reg is destination
    if type(y) == Const:
        testRange(y) 
        putInstr('mov rax, ' + r)
        yc = obtainReg()
        putInstr('mov ' + yc + ', ' + str(y.val))
        putInstr('xor rdx, rdx')
        putInstr('idiv ' + yc)
        releaseReg(yc)
        # remainder is stored in rdx
        putInstr('mov ' + r + ', rdx')

    else:
        if type(y) != Reg: y = loadItem(y)
        putInstr('mov ' + x.reg + ', ' + r)
        putInstr('mov rax, ' + x.reg)
        putInstr('xor rdx, rdx')
        putInstr('idiv ' + y.reg)
        putInstr('mov ' + x.reg + ', rdx')
        releaseReg(y.reg)
    return x



# public functions

def genRec(r):
    """Assuming r is Record, determine fields offsets and the record size"""
    s = 0
    for f in r.fields:
        f.offset, s = s, s + f.tp.size
    r.size = s
    return r

def genArray(a):
    """Assuming r is Array, determine its size"""
    # adds size
    a.size = a.length * a.base.size
    return a

def genLocalVars(sc, start):
    """For list sc of local variables, starting at index starts, determine the
    $fp-relative addresses of variables"""
    s = 0 # local block size
    for i in range(start, len(sc)):
        s = s + sc[i].tp.size
        sc[i].adr = - s
    return s

def genGlobalVars(sc, start):
    """For list sc of global variables, starting at index start, determine the
    address of each variable, which is its name with a trailing _"""
    putInstr('section .bss      ; uninitialized data')
    putInstr('')   
    putLab('number', 'resb 8')   # for reading ints

    for i in range(len(sc) - 1, start - 1, - 1):
        sc[i].adr = sc[i].name + '_'
        putLab(sc[i].adr, 'resb ' + str(sc[i].tp.size))
    putInstr('')
    putInstr('section .text')
    putInstr('')

def progStart():
    putInstr('extern printf')
    putInstr('extern scanf')
    putInstr('global main')
    putInstr('')
    putInstr('section .data')
    putInstr('')
    
    putLab('newline', 'db "", 10, 0')
    putLab('write_msg','db "%d", 10, 0') # format string for printing ints
    putLab('read_msg','db "Enter an integer: ", 0') # message for reading ints
    putLab('read_format','db "%d", 0') # format string for reading ints

    putInstr('')

def progEntry(ident):
    putLab('main')
    putInstr('mov rbx, 0 ; our "zero register"')
    putInstr('')

def progExit(x):
    putInstr('') #newline
    putInstr('mov rax, 60   ;exit call')
    putInstr('mov rdi, 0    ;return code 0')
    putInstr('syscall')
    return asm
        
def procStart():
    global curlev, parblocksize
    curlev = curlev + 1

def genFormalParams(sc):
    """For list sc with formal procedure parameters, determine the $fp-relative
    address of each parameters; each parameter must be type integer, boolean
    or must be a reference parameter"""
    s = 16 # parameter block size
    for p in reversed(sc):
        if p.tp == Int or p.tp == Bool or type(p) == Ref:
            p.adr, s = s, s + 8
        else: mark('no structured value parameters')
    return s

def genProcEntry(ident, parsize, localsize):
    """Declare procedure name, generate code for procedure entry"""
    """ parameters are accessed with EBP + offset """

    putLab(ident)                      
    # set up stack frame
    putInstr('push rbp')
    putInstr('mov rbp, rsp')
    # local variable space
    putInstr('sub rsp, ' + str(localsize))


def genProcExit(x, parsize, localsize): # generates return code
    global curlev
    curlev = curlev - 1

    #restore stack
    putInstr('mov rsp, rbp') 
    putInstr('pop rbp')
    putInstr('ret')
    putInstr('')



def genSelect(x, f):
    # x.f, assuming y is name in one of x.fields
    x.tp, x.adr = f.tp, x.adr + f.offset if type(x.adr) == int else \
                        x.adr + '+' + str(f.offset)
    return x

def genIndex(x, y):
    # x[y], assuming x is ST.Var or ST.Ref, x.tp is ST.Array, y.tp is ST.Int
    # assuming y is Const and y.val is valid index, or Reg integer
    if type(y) == Const:
        offset = (y.val - x.tp.lower) * x.tp.base.size
        x.adr = x.adr + (offset if type(x.adr) == int else ' + ' + str(offset))
    else:
        if type(y) != Reg: y = loadItem(y)

        #pascal arrays have arbitrary indices
        putInstr('sub ' + y.reg + ', ' + str(x.tp.lower))

        putInstr('imul ' + y.reg + ', ' + str(x.tp.base.size))
  
        if x.reg != R0:
            putInstr('add ' + y.reg + ', ' + x.reg)
            releaseReg(x.reg)
        x.reg = y.reg
    x.tp = x.tp.base
    return x

def genVar(x):
    # assuming x is ST.Var, ST.Ref, ST.Const
    # for ST.Const: no code, x.val is constant
    # for ST.Var: x.reg is FP for local, 0 for global vars,
    #   x.adr is relative or absolute address
    # for ST.Ref: address is loaded into register
    # returns ST.Var, ST.Const
    if type(x) == Const: 
        print('constant genvar')
        y = x
    else:
        if x.lev == 0: 
            s = R0
        elif x.lev == curlev: 
            s = FP
        else: mark('level!'); s = R0
        y = Var(x.tp); y.lev = x.lev
        if type(x) == Ref: # reference is loaded into register
            r = obtainReg()
            load(r, s, x.adr)
            y.reg, y.adr = r, 0
        elif type(x) == Var:
            y.reg, y.adr = s, x.adr
        else: y = x # error, pass dummy item
    return y

def genConst(x):
    # assumes x is ST.Const
    return x

def genUnaryOp(op, x):
    """If op is MINUS, NOT, x must be an Int, Bool, and op x is returned.
    If op is AND, OR, x is the first operand (in preparation for the second
    operand"""
    if op == MINUS: # subtract from 0
        if type(x) == Var: x = loadItem(x)
        putInstr('neg ' + x.reg)
    elif op == NOT: # switch condition and branch targets, no code
        if type(x) != Cond: x = loadBool(x)
        x.cond = negate(x.cond) 
        x.labA, x.labB = x.labB, x.labA
    elif op == AND: # load first operand into register and branch
        if type(x) != Cond: x = loadBool(x)

        neg = condOp(negate(x.cond))
        putInstr('cmp ' + x.left + ', ' + x.right)
        putInstr(neg + ' ' + x.labA[0])

        releaseReg(x.left) 
        releaseReg(x.right)
        putLab(x.labB)
    elif op == OR: # load first operand into register and branch
        if type(x) != Cond: x = loadBool(x)
        
        neg = condOp(x.cond)
        putInstr('cmp ' + x.left + ', ' + x.right)
        putInstr(neg + ' ' + x.labB[0])    
        
        releaseReg(x.left); releaseReg(x.right); putLab(x.labA)
    else: assert False
    return x

def genBinaryOp(op, x, y):
    """assumes x.tp == Int == y.tp and op is TIMES, DIV, MOD
    or op is AND, OR"""
    if op == PLUS: 
        y = putOp('add', x, y)
    elif op == MINUS: 
        y = putOp('sub', x, y)
    elif op == TIMES: 
        y = putOp('imul', x, y)
    elif op == DIV: 
        # x64 uses rax register for division
        y = putDivide(x, y)

    elif op == MOD: 
        # use remainder from div op
        y = putModulo(x, y)
    elif op == AND: # load second operand into register 
        if type(y) != Cond: y = loadBool(y)
        y.labA += x.labA # update branch targets
    elif op == OR: # load second operand into register
        if type(y) != Cond: y = loadBool(y)
        y.labB += x.labB # update branch targets
    else: assert False
    return y

def negate(cd):
    """Assume op in {EQ, NE, LT, LE, GT, GE}, return not op"""
    return NE if cd == EQ else \
           EQ if cd == NE else \
           GE if cd == LT else \
           GT if cd == LE else \
           LE if cd == GT else \
           LT

def condOp(cd):
    """Assumes op in {EQ, NE, LT, LE, GT, GE}, return instruction mnemonic"""
    return 'je' if cd == EQ else \
           'jne' if cd == NE else \
           'jl' if cd == LT else \
           'jle' if cd == LE else \
           'jg' if cd == GT else \
           'jge'

def genRelation(op, x, y):
    """Assumes x, y are Int and op is EQ, NE, LT, LE, GT, GE;
    x and y cannot be both constants; return Cond for x op y"""
    if type(x) != Reg: x = loadItem(x)
    if type(y) != Reg: y = loadItem(y)
    return Cond(op, x.reg, y.reg)

assignCount = 0

def genAssign(x, y):
    """Assume x is Var, generate x := y"""
    global assignCount, regs
    if type(y) == Cond: # 
        neg = condOp(negate(y.cond))
        putInstr('cmp ' + y.left + ', ' + y.right)
        putInstr(neg + ' ' + y.labA[0])
        releaseReg(y.left); releaseReg(y.right); r = obtainReg()
        
        putLab(y.labB)  # load true
        putInstr('mov ' + r + ', 1')
        lab = 'A' + str(assignCount); assignCount += 1
        putInstr('jmp ' + lab)
        
        putLab(y.labA) # load false 
        putInstr('mov ' + r + ', 0')
        putLab(lab)
    
    elif type(y) != Reg: y = loadItem(y); r = y.reg
    else: r = y.reg
    store(r, x.reg, x.adr); releaseReg(r)

def genActualPara(ap, fp, n):
    """Pass parameter, ap is actual parameter, fp is the formal parameter,
    either Ref or Var, n is the parameter number"""
    global stacksize

    if type(fp) == Ref:  #  reference parameter, assume p is Var

        if ap.adr != 0:  #  load address in register
            r = obtainReg()
            loadAddress(r, ap.reg, ap.adr)
        else: 
            r = ap.reg  #  address already in register
        
        putInstr('push ' + r)
        stacksize += 8 # we restore stack after return
        releaseReg(r)

    else:  #  value parameter
        if type(ap) != Cond:
            if type(ap) != Reg: 
                ap = loadItem(ap)
            putInstr('push ' + ap.reg)
            stacksize += 8
            releaseReg(ap.reg)
        else: mark('unsupported parameter type')

def genCall(pr):
    """Assume pr is Proc"""
    global stacksize

    putInstr('call ' + pr.name)
    putInstr('add rsp, ' + str(stacksize))
    stacksize = 0

    
# Use C scanf, printf calls for I/O

def genRead(x):
    """Assume x is Var"""

    putInstr('')
    putInstr('mov rdi, read_msg')
    putInstr('mov rax, 0')
    putInstr('call printf')
    putInstr('mov rdi, read_format')
    putInstr('mov rsi, number')
    putInstr('mov rax, 0')
    putInstr('call scanf')
    putInstr('mov rsi, [number]')
    putInstr('mov [' + str(x.adr) + '], rsi')
    putInstr('')


def genWrite(x):

    # use c printf 
    putInstr('')
    putInstr('mov rdi, write_msg')
    loadItemReg(x, 'rsi')
    putInstr('mov rax, 0')
    putInstr('call printf')
    putInstr('')


def genWriteln():

    putInstr('')
    putInstr('mov rdi, newline')
    putInstr('mov rax, 0')
    putInstr('call printf')
    putInstr('')


def genSeq(x, y):
    """Assume x and y are statements, generate x ; y"""
    pass

def genCond(x):
    """Assume x is Bool, generate code for branching on x"""
    if type(x) != Cond: x = loadBool(x)
    neg = condOp(negate(x.cond))
    putInstr('cmp ' + x.left + ', ' + x.right)
    putInstr(neg + ' ' + x.labA[0])
    releaseReg(x.left); releaseReg(x.right); putLab(x.labB)
    return x

def genIfThen(x, y):
    """Generate code for if-then: x is condition, y is then-statement"""
    putLab(x.labA)

ifCount = 0

def genThen(x, y):
    """Generate code for if-then-else: x is condition, y is then-statement"""
    global ifCount
    lab = 'I' + str(ifCount); ifCount += 1
    putInstr('jmp ' + lab)
    putLab(x.labA); 
    return lab

def genIfElse(x, y, z):
    """Generate code of if-then-else: x is condition, y is then-statement,
    z is else-statement"""
    putLab(y)

loopCount = 0

def genTarget():
    """Return target for loops with backward branches"""
    global loopCount
    lab = 'L' + str(loopCount); loopCount += 1
    putLab(lab)
    return lab

def genWhile(t, x, y):
    """Generate code for while: t is target, x is condition, y is body"""
    putInstr('jmp ' + t)
    putLab(x.labA); 



In [ ]:
# %load P0.py
"""
Pascal0 Parser, Emil Sekerinski, March 2016,
Main program, type-checks, folds constants, calls scanner SC and code
generator CG, uses symbol table ST
"""

from sys import argv
import SC  #  used for SC.init, SC.sym, SC.val
from SC import TIMES, DIV, MOD, AND, PLUS, MINUS, OR, EQ, NE, LT, GT, \
     LE, GE, PERIOD, COMMA, COLON, RPAREN, RBRAK, OF, THEN, DO, LPAREN, \
     LBRAK, NOT, BECOMES, NUMBER, IDENT, SEMICOLON, END, ELSE, IF, WHILE, \
     ARRAY, RECORD, CONST, TYPE, VAR, PROCEDURE, BEGIN, PROGRAM, EOF, \
     getSym, mark
import ST  #  used for ST.init
from ST import Var, Ref, Const, Type, Proc, StdProc, Int, Bool,  Record, \
     Array, newObj, find, openScope, topScope, closeScope
import CGx86 as CG  #  used for CG.init
from CGx86 import genRec, genArray, progStart, genGlobalVars, progEntry, \
     progExit, procStart, genFormalParams, genActualPara, genLocalVars, \
     genProcEntry, genProcExit, genSelect, genIndex, genVar, genConst, \
     genUnaryOp, genBinaryOp, genRelation, genSeq, genAssign, genCall, \
     genRead, genWrite, genWriteln, genCond, genIfThen, genThen, genIfElse, \
     genTarget, genWhile

# first and follow sets for recursive descent parsing

FIRSTFACTOR = {IDENT, NUMBER, LPAREN, NOT}
FOLLOWFACTOR = {TIMES, DIV, MOD, AND, OR, PLUS, MINUS, EQ, NE, LT, LE, GT,
                GE, COMMA, SEMICOLON, THEN, ELSE, RPAREN, DO, PERIOD, END}
FIRSTEXPRESSION = {PLUS, MINUS, IDENT, NUMBER, LPAREN, NOT}
FIRSTSTATEMENT = {IDENT, IF, WHILE, BEGIN}
FOLLOWSTATEMENT = {SEMICOLON, END, ELSE, BEGIN}
FIRSTTYPE = {IDENT, RECORD, ARRAY, LPAREN}
FOLLOWTYPE = {SEMICOLON}
FIRSTDECL = {CONST, TYPE, VAR, PROCEDURE}
FOLLOWDECL = {BEGIN, END, PROCEDURE, EOF}
FOLLOWPROCCALL = {SEMICOLON, END, ELSE, IF, WHILE}
STRONGSYMS = {CONST, TYPE, VAR, PROCEDURE, WHILE, IF, BEGIN, EOF}

# parsing procedures

def selector(x):
    """
    Parses
        selector = {"." ident | "[" expression "]"}.
    Assumes x is the entry for the identifier in front of the selector;
    generates code for the selector if no error is reported
    """
    while SC.sym in {PERIOD, LBRAK}:
        if SC.sym == PERIOD:  #  x.f
            getSym()
            if SC.sym == IDENT:
                if type(x.tp) == Record:
                    for f in x.tp.fields:
                        if f.name == SC.val:
                            x = genSelect(x, f); break
                    else: mark("not a field")
                    getSym()
                else: mark("not a record")
            else: mark("identifier expected")
        else:  #  x[y]
            getSym(); y = expression()
            if type(x.tp) == Array:
                if y.tp == Int:
                    if type(y) == Const and \
                       (y.val < x.tp.lower or y.val >= x.tp.lower + x.tp.length):
                        mark('index out of bounds')
                    else: x = genIndex(x, y)
                else: mark('index not integer')
            else: mark('not an array')
            if SC.sym == RBRAK: getSym()
            else: mark("] expected")
    return x

def factor():
    """
    Parses
        factor = ident selector | integer | "(" expression ")" | "not" factor.
    Generates code for the factor if no error is reported
    """
    if SC.sym not in FIRSTFACTOR:
        mark("factor expected"); getSym()
        while SC.sym not in FIRSTFACTOR | STRONGSYMS | FOLLOWFACTOR:
            getSym()
    if SC.sym == IDENT:
        x = find(SC.val)
        if type(x) in {Var, Ref}: x = genVar(x)
        elif type(x) == Const: x = Const(x.tp, x.val); x = genConst(x)
        else: mark('variable or constant expected')
        getSym(); x = selector(x)
    elif SC.sym == NUMBER:
        x = Const(Int, SC.val); x = genConst(x); getSym()
    elif SC.sym == LPAREN:
        getSym(); x = expression()
        if SC.sym == RPAREN: getSym()
        else: mark(") expected")
    elif SC.sym == NOT:
        getSym(); x = factor()
        if x.tp != Bool: mark('not boolean')
        elif type(x) == Const: x.val = 1 - x.val # constant folding
        else: x = genUnaryOp(NOT, x)
    else:
        mark("factor expected"); x = None
    return x

def term():
    """
    Parses
        term = factor {("*" | "div" | "mod" | "and") factor}.
    Generates code for the term if no error is reported
    """
    x = factor()
    while SC.sym in {TIMES, DIV, MOD, AND}:
        op = SC.sym; getSym();
        if op == AND and type(x) != Const: x = genUnaryOp(AND, x)
        y = factor() # x op y
        if x.tp == Int == y.tp and op in {TIMES, DIV, MOD}:
            if type(x) == Const == type(y): # constant folding
                if op == TIMES: x.val = x.val * y.val
                elif op == DIV: x.val = x.val // y.val
                elif op == MOD: x.val = x.val % y.val
            else: x = genBinaryOp(op, x, y)
        elif x.tp == Bool == y.tp and op == AND:
            if type(x) == Const: # constant folding
                if x.val: x = y # if x is true, take y, else x
            else: x = genBinaryOp(AND, x, y)
        else: mark('bad type')
    return x

def simpleExpression():
    """
    Parses
        simpleExpression = ["+" | "-"] term {("+" | "-" | "or") term}.
    Generates code for the simpleExpression if no error is reported
    """
    if SC.sym == PLUS:
        getSym(); x = term()
    elif SC.sym == MINUS:
        getSym(); x = term()
        if x.tp != Int: mark('bad type')
        elif type(x) == Const: x.val = - x.val # constant folding
        else: x = genUnaryOp(MINUS, x)
    else: x = term()
    while SC.sym in {PLUS, MINUS, OR}:
        op = SC.sym; getSym()
        if op == OR and type(x) != Const: x = genUnaryOp(OR, x)
        y = term() # x op y
        if x.tp == Int == y.tp and op in {PLUS, MINUS}:
            if type(x) == Const == type(y): # constant folding
                if op == PLUS: x.val = x.val + y.val
                elif op == MINUS: x.val = x.val - y.val
            else: x = genBinaryOp(op, x, y)
        elif x.tp == Bool == y.tp and op == OR:
            if type(x) == Const: # constant folding
                if not x.val: x = y # if x is false, take y, else x
            else: x = genBinaryOp(OR, x, y)
        else: mark('bad type')
    return x

def expression():
    """
    Parses
        expression = simpleExpression
                     {("=" | "<>" | "<" | "<=" | ">" | ">=") simpleExpression}.
    Generates code for the expression if no error is reported
    """
    x = simpleExpression()
    while SC.sym in {EQ, NE, LT, LE, GT, GE}:
        op = SC.sym; getSym(); y = simpleExpression() # x op y
        if x.tp == Int == y.tp:
            x = genRelation(op, x, y)
        else: mark('bad type')
    return x

def compoundStatement():
    """
    Parses
        compoundStatement = "begin" statement {";" statement} "end"
    Generates code for the compoundStatement if no error is reported
    """
    if SC.sym == BEGIN: getSym()
    else: mark("'begin' expected")
    x = statement()
    while SC.sym == SEMICOLON or SC.sym in FIRSTSTATEMENT:
        if SC.sym == SEMICOLON: getSym()
        else: mark("; missing")
        y = statement(); x = genSeq(x, y)
    if SC.sym == END: getSym()
    else: mark("'end' expected")
    return x

def statement():
    """
    Parses
        statement = ident selector ":=" expression |
                    ident "(" [expression {"," expression}] ")" |
                    compoundStatement |
                    "if" expression "then" Statement ["else" Statement] |
                    "while" expression "do" Statement.
    Generates code for the statement if no error is reported
    """
    if SC.sym not in FIRSTSTATEMENT:
        mark("statement expected"); getSym()
        while SC.sym not in FIRSTSTATEMENT | STRONGSYMS | FOLLOWSTATEMENT:
            getSym()
    if SC.sym == IDENT:
        x = find(SC.val); getSym(); x = genVar(x)
        if type(x) in {Var, Ref}:
            x = selector(x)
            if SC.sym == BECOMES:
                getSym(); y = expression()
                if x.tp == y.tp in {Bool, Int}:
                    if type(x) == Var: x = genAssign(x, y)
                    else: mark('illegal assignment')
                else: mark('incompatible assignment')
            elif SC.sym == EQ:
                mark(':= expected'); getSym(); y = expression()
            else: mark(':= expected')
        elif type(x) in {Proc, StdProc} and SC.sym == LPAREN:
            getSym()
            fp, i = x.par, 0  #  list of formal parameters
            if SC.sym in FIRSTEXPRESSION:
                y = expression()
                if i < len(fp):
                    if type(fp[i]) == Var or type(y) == Var: # fp[i] == Ref and ty
                        if type(x) == Proc: genActualPara(y, fp[i], i)
                        i = i + 1
                    else: mark('illegal parameter mode')
                else: mark('extra parameter')
                while SC.sym == COMMA:
                    getSym()
                    y = expression()
                    if i < len(fp):
                        if type(fp[i]) == Var or type(y) == Var:
                            if type(x) == Proc: genActualPara(y, fp[i], i)
                            i = i + 1
                        else: mark('illegal parameter mode')
                    else: mark('extra parameter')
            if i < len(fp): mark('too few parameters')
            #if i > 0: mark('too few parameters')
            if SC.sym == RPAREN: getSym()
            else: mark("')' expected")
            if type(x) == StdProc:
                if x.name == 'read': x = genRead(y)
                elif x.name == 'write': x = genWrite(y)
                elif x.name == 'writeln': x = genWriteln()
            else: x = genCall(x)
        else: mark("variable or procedure expected")
    elif SC.sym == BEGIN: x = compoundStatement()
    elif SC.sym == IF:
        getSym(); x = expression();
        if x.tp == Bool: x = genCond(x)
        else: mark('boolean expected')
        if SC.sym == THEN: getSym()
        else: mark("'then' expected")
        y = statement()
        if SC.sym == ELSE:
            y = genThen(x, y); getSym(); z = statement();
            x = genIfElse(x, y, z)
        else: x = genIfThen(x, y)
    elif SC.sym == WHILE:
        getSym(); t = genTarget(); x = expression()
        if x.tp == Bool: x = genCond(x)
        else: mark('boolean expected')
        if SC.sym == DO: getSym()
        else: mark("'do' expected")
        y = statement(); x = genWhile(t, x, y)
    else:
        mark('invalid statement'); x = None
    return x

def typ():
    """
    Parses
        type = ident |
               "array" "[" expression ".." expression "]" "of" type |
               "record" typedIds {";" typedIds} "end".
    Returns a type descriptor 
    """
    if SC.sym not in FIRSTTYPE:
        getSym(); mark("type expected")
        while SC.sym not in FIRSTTYPE | FOLLOWTYPE |STRONGSYMS:
            getSym()
    if SC.sym == IDENT:
        ident = SC.val; x = find(ident); getSym()
        if type(x) == Type: x = Type(x.tp)
        else: mark('not a type')
    elif SC.sym == ARRAY:
        getSym()
        if SC.sym == LBRAK: getSym()
        else: mark("'[' expected")
        x = expression()
        if type(x) != Const or x.val < 0: mark('bad lower bound')
        if SC.sym == PERIOD: getSym()
        else: mark("'.' expected")
        if SC.sym == PERIOD: getSym()
        else: mark("'.' expected")
        y = expression()
        if type(y) != Const or y.val < x.val: mark('bad upper bound')
        if SC.sym == RBRAK: getSym()
        else: mark("']' expected")
        if SC.sym == OF: getSym()
        else: mark("'of' expected")
        z = typ().tp; l = y.val - x.val + 1
        x = Type(genArray(Array(z, x.val, l)))
    elif SC.sym == RECORD:
        getSym(); openScope(); typedIds(Var)
        while SC.sym == SEMICOLON:
            getSym(); typedIds(Var)
        if SC.sym == END: getSym()
        else: mark("'end' expected")
        r = topScope(); closeScope()
        x = Type(genRec(Record(r)))
    else: mark("type expected"); x = Type(None)
    return x

def typedIds(kind):
    """
    Parses
        typedIds = ident {"," ident} ":" type.
    Updates current scope of symbol table
    Assumes kind is Var or Ref and applies it to all identifiers
    Reports an error if an identifier is already defined in the current scope
    """
    tid = [SC.val]; getSym()
    while SC.sym == COMMA:
        getSym()
        if SC.sym == IDENT: tid.append(SC.val); getSym()
        else: mark('identifier expected')
    if SC.sym == COLON:
        getSym(); tp = typ().tp
        for i in tid: newObj(i, kind(tp))
    else: mark("':' expected")

def declarations(allocVar):
    """
    Parses
        declarations =
            {"const" ident "=" expression ";"}
            {"type" ident "=" type ";"}
            {"var" typedIds ";"}
            {"procedure" ident ["(" [["var"] typedIds {";" ["var"] typedIds}] ")"] ";"
                declarations compoundStatement ";"}.
    Updates current scope of symbol table.
    Reports an error if an identifier is already defined in the current scope.
    For each procedure, code is generated
    """
    if SC.sym not in FIRSTDECL | FOLLOWDECL:
        getSym(); mark("declaration expected")
        while SC.sym not in FIRSTDECL | FOLLOWDECL: getSym()
    while SC.sym == CONST:
        getSym()
        if SC.sym == IDENT:
            ident = SC.val; getSym()
            if SC.sym == EQ: getSym()
            else: mark("= expected")
            x = expression()
            if type(x) == Const: newObj(ident, x)
            else: mark('expression not constant')
        else: mark("constant name expected")
        if SC.sym == SEMICOLON: getSym()
        else: mark("; expected")
    while SC.sym == TYPE:
        getSym()
        if SC.sym == IDENT:
            ident = SC.val; getSym()
            if SC.sym == EQ: getSym()
            else: mark("= expected")
            x = typ(); newObj(ident, x)  #  x is of type ST.Type
            if SC.sym == SEMICOLON: getSym()
            else: mark("; expected")
        else: print(SC.sym); mark("type name expected")
    start = len(topScope())
    while SC.sym == VAR:
        getSym(); typedIds(Var)
        if SC.sym == SEMICOLON: getSym()
        else: mark("; expected")
    varsize = allocVar(topScope(), start)
    while SC.sym == PROCEDURE:
        getSym()
        if SC.sym == IDENT: getSym()
        else: mark("procedure name expected")
        ident = SC.val; newObj(ident, Proc([])) #  entered without parameters
        sc = topScope()
        procStart(); openScope() # new scope for parameters and body
        if SC.sym == LPAREN:
            getSym()
            if SC.sym in {VAR, IDENT}:
                if SC.sym == VAR: getSym(); typedIds(Ref) #, procParams)
                else: typedIds(Var) #, procParams)
                while SC.sym == SEMICOLON:
                    getSym()
                    if SC.sym == VAR: getSym(); typedIds(Ref) #, procParams)
                    else: typedIds(Var) #, procParams)
            else: mark("formal parameters expected")
            fp = topScope()
            sc[-1].par = fp[:] #  procedure parameters updated
            if SC.sym == RPAREN: getSym()
            else: mark(") expected")
        else: fp = []
        parsize = genFormalParams(fp)
        if SC.sym == SEMICOLON: getSym()
        else: mark("; expected")
        # PROCEDURE CALL
        localsize = declarations(genLocalVars)
        genProcEntry(ident, parsize, localsize)
        x = compoundStatement() 
        genProcExit(x, parsize, localsize)
        closeScope() #  scope for parameters and body closed
        if SC.sym == SEMICOLON: getSym()
        else: mark("; expected")
    return varsize

def program():
    """
    Parses
        program = "program" ident ";" declarations compoundStatement.
    Generates code if no error is reported
    """
    newObj('boolean', Type(Bool)); Bool.size = 8 # 64 bit sizes
    newObj('integer', Type(Int)); Int.size = 8
    newObj('true', Const(Bool, 1))
    newObj('false', Const(Bool, 0))
    newObj('read', StdProc([Ref(Int)]))
    newObj('write', StdProc([Var(Int)]))
    newObj('writeln', StdProc([]))
    progStart()
    if SC.sym == PROGRAM: getSym()
    else: mark("'program' expected")
    ident = SC.val
    if SC.sym == IDENT: getSym()
    else: mark('program name expected')
    if SC.sym == SEMICOLON: getSym()
    else: mark('; expected')
    declarations(genGlobalVars); progEntry(ident)
    x = compoundStatement()
    return progExit(x)

def compileString(src, dstfn = None):
    """Compiles string src; if dstfn is provided, the code is written to that
    file, otherwise printed on the screen"""
    SC.init(src)
    ST.init()
    CG.init()
    p = program()
    if dstfn == None: print(p)
    else:
        with open(dstfn, 'w') as f: f.write(p);

def compileFile(srcfn):
    if srcfn.endswith('.p'):
        with open(srcfn, 'r') as f: src = f.read()
        dstfn = srcfn[:-2] + '.s'
        compileString(src, dstfn)
    else: mark("'.p' file extension expected")

# sampe usage:
# import os
# os.chdir('/Users/emil/Google Drive/CS 4TB4 - Winter 2016/Assignment 3')
# compileFile('disassembly.p')


## Appendix B: Compiler Script
#### compile.p

In [ ]:
# %load compile.py
import os, sys

from P0 import compileString

#python3 Compile.py tests/filename

def compile(srcfn):

    if srcfn.endswith('.p'):
        with open(srcfn, 'r') as f: src = f.read()
        dstfn = srcfn[:-2] + '.s'
        objfn = srcfn[:-2] + '.o'
        binfn = srcfn[:-2]
        compileString(src, dstfn)

        os.system('nasm -f elf64 '+ dstfn + ' && gcc -m64 -o ' + binfn + ' ' + objfn)
        os.system('rm ' + objfn)

    else: print("'.p' file extension expected")

if __name__ == "__main__":
  compile(sys.argv[1])


## Appendix C: Test Programs

#### readwrite.p
Test read, write, and writeln function

In [ ]:
# %load tests/readwrite.p
program p;
  var x: integer;
  begin 
    read(x);
    x := 3 * x;
    write(x);
    write(x * 5);
    writeln();
    read(x);
    write(x + 5)
  end;
  {for input 0, 0 write 0, 0, 5}
  {for input 5, -3 write 15, 75, 2}

#### arthimetic.p
Test order of operations, constant operations, operations with 1 and 0.

In [ ]:
# %load tests/arithmetic.p
program p;
  var w,x,y,z: integer;
  begin 

    {multiply, divide by one, zero}
    x := 5 * 1;
    write(x);
    x := 5 div 1;
    write(x);
    x := x * 0 * 3; {write 5 5 0}
    write(x);
    writeln();

    {add, multiply, divide, modulo by constant}
    
    w := 30 div 3;
    w := w div 2;
    x := 23 mod 5;
    x := x mod 2;
    y := 3 + 4;
    y := y + 3;
    z := 5 * 2;
    z := z * 3;
    write(w);write(x);
    write(y);write(z); {write 5 1 10 30}
    writeln();

    {order of operations}
    x := w * x + z div y; {write 8}
    writeln() ;

    {negation}
    x := -x + x;
    y := -y + z;
    write(x);write(y); {write 0 20 }
    writeln()

  end;

#### arrays_records.p
Test arrays and records.

In [ ]:
# %load tests/arrays_records.p
program p;
  type a = array [1 .. 7] of integer;
  type r = record f: integer; g: a; h: integer end;
  var v: a;
  var w: r;
  var x: integer;
  procedure q(var c: a; var d: r);
    var y: integer;
    begin y := 3;
      write(d.h); write(c[1]); write(d.g[y]); {writes 5, 3, 9}
      writeln(); c[7] := 7; write(c[y+4]); {writes 7}
      d.g[y*2] := 7; write(d.g[6]) {writes 7}
    end;
  begin x := 9;
    w.h := 12 - 7; write(w.h); {writes 5}
    v[1] := 3; write(v[x-8]); {writes 3}
    w.g[x div 3] := 9; write(w.g[3]); {writes 9}
    writeln(); q(v, w); writeln();
    write(v[7]); write(w.g[6]) {writes 7, 7}
  end;

#### order.p
Test correct ordering of passed arguments.

In [ ]:
# %load tests/order.p
program p;
  var xx, xy, xz, xp: integer;
  procedure q(var a, b, c, d: integer);
    var x,y,z,p: integer;
    begin 
      x := a;
      y := b;
      z := c;
      p := d;
      write(x);
      write(xy);
      write(z);
      write(xp)
    end;
  begin 
    xx := 1;
    xy := 2;
    xz := 3;
    xp := 4;
    q(xx, xy, xz, xp) {write 1 2 3 4}
  end;

#### conditions.p
Test conditional statements, while loop, constants

In [ ]:
# %load tests/conditions.p
program p;
  const five = 5;
  const seven = 7;
  const always = true;
  const never = false;
  var x, y, z: integer;
  var b, t, f: boolean;
  begin x := seven; y := 9; z := 11; t := true; f := false;
    if true then write(7) else write(9);    {writes 7}
    if false then write(7) else write(9);   {writes 9}
    if t then write(7) else write(9);       {writes 7}
    if f then write(7) else write(9);       {writes 9}
    if not t then write(7) else write(9);   {writes 9}
    if not f then write(7) else write(9);   {writes 7}
    if t or t then write(7) else write(9);  {writes 7}
    if t or f then write(7) else write(9);  {writes 7}
    if f or t then write(7) else write(9);  {writes 7}
    if f or f then write(7) else write(9);  {writes 9}
    if t and t then write(7) else write(9); {writes 7}
    if t and f then write(7) else write(9); {writes 9}
    if f and t then write(7) else write(9); {writes 9}
    if f and f then write(7) else write(9); {writes 9}
    writeln();
    b := true;
    if b then write(3) else write(5); {writes 3}
    b := false;
    if b then write(3) else write(5); {writes 5}
    b := x < y;
    if b then write(x) else write(y); {writes 7}
    b := (x > y) or t;
    if b then write(3) else write(5); {writes 3}
    b := (x > y) or f;
    if b then write(3) else write(5); {writes 5}
    b := (x = y) or (x > y);
    if b then write(3) else write(5); {writes 5}
    b := (x = y) or (x < y);
    if b then write(3) else write(5); {writes 3}
    b := f and (x >= y);
    if b then write(3) else write(5); {writes 5}
    writeln();
    while y > 3 do                    {writes 9, 8, 7, 6, 5, 4}
      begin write(y); y := y - 1 end;
    write(y); writeln();              {writes 3}
    if not(x < y) and t then          {writes 7}
      write(x)
  end;

#### params.p
Test parameter passing.

In [ ]:
# %load tests/params.p
program p;
  var x: integer;
  procedure q({-4($sp)}a: integer {4($fp)}; {-8($sp)}var b: integer {($fp)});
    var y: integer;{-12($fp)}
    begin y := a; write(y); writeln(); {writes 7}
      a := b; write(x); write(a); writeln(); {writes 5, 5}
      b := y; write(b); write(x); writeln(); {writes 7, 7}
      write(a); write(y); writeln() {writes 5, 7}
    end;
  begin x := 5; q(7, x);
    write(x) {writes 7}
  end;

#### scopes.p
Additional combined testing.

In [ ]:
# %load tests/scopes.p
program p;
  const seven = (9 mod 3 + 5 * 3) div 2;
  type int = integer;
  var x, y: integer;
  procedure q;
    const sotrue = true and true;
    const sofalse = false and true;
    const alsotrue = false or true;
    const alsofalse = false or false;
    var x: int;
    begin x := 3;
      if sotrue then y := x else y := seven;
      write(y); {writes 3}
      if sofalse then y := x else y := seven;
      write(y); {writes 7}
      if alsotrue then y := x else y := seven;
      write(y); {writes 3}
      if alsofalse then y := x else y := seven;
      write(y); {writes 7}
      if not(true or false) then write(5) else write(9)
    end;
  begin x := 7; q(); write(x) {writes 9 7}
  end;

#### codegen.p
Additional combined testing

In [ ]:
# %load tests/codegen.p
program p;
  var g: integer;          {global variable}
  procedure q(v: integer); {value parameter}
    var l: integer;        {local variable}
    begin
      l := 9;
      if l > v then
         write(l)
      else
         write(g)
    end;
  begin
    g := 5;
    q(7)  {write 9}
  end;

#### factorial.p
Compute factorial up to user input.

In [ ]:
# %load tests/factorial.p
program p;
  const size = 10;
  type arr = array [1 .. 25] of integer;
  var i, j, factorial: integer;
  var a1, a2: arr;
  procedure writestuff(var a, b: integer);
    var aaa,bbb,ccc,ddd,eee,fff,ggg: integer;
    begin 
      eee := 42;
      ggg := 11;
      write(a);
      write(b);
      writeln()
    end;
  begin 
    i := 1;
    factorial := 1;
    read(j);
    while i <= j do
      begin 
        factorial := factorial * i;
        a1[i] := factorial;
        writestuff(i, a1[i]);
        i := i + 1
      end
  end;

{
Expected output for input 12:

1
1

2
2

3
6

4
24

5
120

6
720

7
5040

8
40320

9
362880

10
3628800

11
39916800

12
479001600
}